In [ ]:
!pip install pandas kaggle
!pip install keras tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/



 ## Metodologia

*   Definição do problema
*   Dados do problema
*   Carregamento dos dados
*   Analise e pré-processamento dos dados
*   Modelo







## 1. Carregamento dos Dados

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import keras

In [ ]:
# Carregar o conjunto de dados
data = pd.read_csv('/content/sample_data/cod.csv')

# Verificar as primeiras linhas do conjunto de dados
data.head()


,name,wins,kills,kdRatio,killstreak,level,losses,prestige,hits,timePlayed,headshots,averageTime,gamesPlayed,assists,misses,xp,scorePerMinute,shots,deaths
0,RggRt45#4697369,0,0,0.00000,0,1,0,0,0,0,0,0.0,0,0,0,0,0.0,0,0
1,JohniceRex#9176033,0,0,0.00000,0,1,0,110,0,7,0,7.0,0,0,0,700,0.0,0,16
2,bootybootykill#1892064,0,66,1.03125,0,9,0,110,0,32,16,32.0,0,1,0,48300,0.0,0,64
3,JNaCo#5244172,3,2,0.40000,0,1,0,0,0,3,0,3.0,0,0,0,1150,0.0,0,5
4,gomezyayo_007#6596687,0,2,0.20000,0,1,0,110,0,5,1,5.0,0,0,0,1000,0.0,0,10


##2. Análise e Pré-processamento dos Dados


##2.1 Retirando dados que não serão usados
Não será necessário realizar etapas de pré-processamentos, pois as colunas selecionadas não possuem valores nulos

In [57]:
# Carregar o dataset
dataset = pd.read_csv('/content/sample_data/cod.csv')
#Remover uma coluna específica
dataset.drop(['name', 'kills', 'killstreak', 'level', 'losses',
       'prestige', 'hits', 'timePlayed', 'headshots', 'averageTime',
       'gamesPlayed', 'assists', 'misses', 'xp', 'scorePerMinute', 'shots',
       'deaths'], axis=1, inplace=True)

dataset.columns

Index(['wins', 'kdRatio'], dtype='object')

##2.2 Visualização dos dados

In [58]:
num = dataset
print(num.columns)

Index(['wins', 'kdRatio'], dtype='object')


##3 Modelo



#3.1 Criando uma coluna

In [61]:
##Criando uma nova coluna com valores binários, onde 1 indica uma vitoria e 0 indica uma derrota
data['winProbability'] = data['wins'].apply(lambda x: 1 if x > 0 else 0)

In [62]:
# Verificar as estatísticas básicas das colunas relevantes
print(data[['kdRatio', 'wins', 'winProbability']].describe())

           kdRatio         wins  winProbability
count  1558.000000  1558.000000     1558.000000
mean      0.637098   153.002567        0.660462
std       0.430459   301.984766        0.473704
min       0.000000     0.000000        0.000000
25%       0.261442     0.000000        0.000000
50%       0.732837    10.000000        1.000000
75%       0.955293   168.000000        1.000000
max       3.000000  3519.000000        1.000000


#3.2 Separando os dados para treino e teste

In [63]:
x = data[['kdRatio']]
y = data['winProbability']

In [64]:
# X shape contem numero linhas e colunas do dataframe X
# Y shape numeros de elementos que voce tem no Y (1)
#Data frame do X possui 1558 linhas e 1 colunas e no Y ele possui 1558 elementos
print(x.shape)
print(y.shape)

(1558, 1)
(1558,)


In [65]:
from sklearn.model_selection import train_test_split

# divide o dataset de treinamento em treinamento e teste, separando 30% em teste

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [66]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(1090, 1) (1090,) (468, 1) (468,)


## 3.3 Crianddo o modelo
   Árvore de Decisão Simples

In [67]:
from sklearn.tree import DecisionTreeClassifier
# cria a arvore
tree = DecisionTreeClassifier(criterion='gini', splitter='random', max_depth=3, random_state=0)
#cria o modelo
model = tree.fit(x_train, y_train)

In [68]:
from sklearn.metrics import accuracy_score
# tenta predicao dos dados de teste
predict = model.predict(x_test)

In [69]:
accuracy = accuracy_score(y_test, predict)
print('Acurácia: {:.2f}%'.format(accuracy * 100))

Acurácia: 81.62%


In [77]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(units=108, kernel_initializer='uniform', activation='relu', input_dim=1))
model.add(Dropout(rate=0.1))
model.add(Dense(units=108, kernel_initializer='uniform', activation='relu'))
model.add(Dropout(rate=0.1))
model.add(Dense(units=108, kernel_initializer='uniform', activation='relu'))
model.add(Dropout(rate=0.1))
model.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=100, epochs=400, validation_data=(x_test, y_test))

Epoch 1/400
11/11 [==============================] - 3s 40ms/step - loss: 0.6887 - accuracy: 0.6872 - val_loss: 0.6815 - val_accuracy: 0.6453
Epoch 2/400
11/11 [==============================] - 0s 8ms/step - loss: 0.6633 - accuracy: 0.6670 - val_loss: 0.6404 - val_accuracy: 0.6453
Epoch 3/400
11/11 [==============================] - 0s 10ms/step - loss: 0.6029 - accuracy: 0.6670 - val_loss: 0.5965 - val_accuracy: 0.6453
Epoch 4/400
11/11 [==============================] - 0s 8ms/step - loss: 0.5651 - accuracy: 0.6670 - val_loss: 0.5739 - val_accuracy: 0.6453
Epoch 5/400
11/11 [==============================] - 0s 28ms/step - loss: 0.5328 - accuracy: 0.6670 - val_loss: 0.5377 - val_accuracy: 0.6453
Epoch 6/400
11/11 [==============================] - 0s 26ms/step - loss: 0.4965 - accuracy: 0.7431 - val_loss: 0.5172 - val_accuracy: 0.8162
Epoch 7/400
11/11 [==============================] - 0s 20ms/step - loss: 0.4879 - accuracy: 0.8229 - val_loss: 0.5159 - val_accuracy: 0.8141
Epoch 8/

In [78]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Erro:', score[0])
print('Precisão:', score[1]*100, "%" )

Erro: 0.4218652844429016
Precisão: 81.83760643005371 %
